In [ ]:
import pandas as pd
import numpy as np

twitter_df=pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
twitter_df.head()

In [ ]:
twitter_df.shape

In [ ]:
import requests

predictions=requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [ ]:
predictions.headers['content-type']

In [ ]:
open('image_predictions.csv','wb').write(predictions.content)

In [ ]:
prediction_df=pd.read_csv('image_predictions.tsv',sep='/t')

In [ ]:
import tweepy

consumer_key = '0ef9ZN1lehgc3G4rh0SQvyYZA'
consumer_secret = '6dA51cggFj5XqbFLG9PvFGfSYhPUHSQQyHkJnK7ddaFtkP3O6q'
access_token = 'sHB4eFlSsid2HhjDo16qQRMSvCaoTE'
access_secret = 'yXIRsIeo3xkWXzLobFLiPLjZMXNfmF8dHgu9HGd9LK01e'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
import json
tweets=[]
for line in open('tweet_json.txt','r'):
    try:
        tweets.append(json.loads(line))
    except ValueError:
        pass

In [ ]:
len(tweets)

In [ ]:
import pandas as pd

tweets=pd.DataFrame(tweets)

In [ ]:
tweets.info()

I join the twitter_df table with the tweets table. I use a left join, so that all the rows in twitter_df are preserved, but the tweet_ids that are not in twitter_df are removed. I only added the following columns from tweets: 'retweet_count','created_at','favorite_count','entities'.

In [ ]:
tweet_df=twitter_df.join(tweets[['retweet_count','created_at','favorite_count','entities']])

Let's take a look at what is in the 'entities' column:

In [ ]:
tweet_df['entities'][0]

Next, I extract some data from the 'entities' column. To begin with, I want to extract `hashtags`, `symbols`, `urls`, and `user_mentions`. I'll decide what to keep from these after I take a look at the data.

In [ ]:
def hashtag_to_string(dic):
    st=''
    for hashtag in dic['hashtags']:
        st+=str(hashtag['text'])+','
    return st

i=0
for i in range (0,tweet_df.shape[0]):
    try:
        tweet_df.loc[i,'hashtags']=hashtag_to_string(tweet_df.loc[i,'entities'])
        i=i+1
    except TypeError:
        print i
        pass
    except KeyError:
        tweet_df.loc[i,'hashtags']=' '

tweet_df.head()

In [ ]:
tweets[tweets['in_reply_to_user_id'].notnull()].iloc[:,1:18]

Add: retweet_count,created_at,favorite_count

Cleaning:
1. Names: replace those starting with lowercase with 'None'
2. Ratings: normalize to denominator 10: multiply numerator by 10, divide by denominator 
3. Check to see if any dog categories were missed.
4. Remove rows that are retweets. Use any of the 'retweeted' columns to identify these (if any are left after merging?)

